# Feature Engineering 1
* * *
这个 **FE** 中，经过之前的 **DEA** 分析，我们可以看到new与his两个文件的区别，仅仅是以2018年2月为节点，在此之前的归入his文件，在此之后的划入new中，所以，我们可以合并两个文件。并且，针对这个新的数据集，我们将作以下变换：         
**authorized_flag**我们将bool化该特征，并求出其count与mean。因为其值的不同预示着交易是否成功，所以推荐将 'N' 与 'Y' 分开成两个数据集。   

**city_id** nunqiue, mode   
**merchant_category_id** nunique，mode          
**merchant_id** nunique，mode          
**state_id** nunique, mode
**subsector_id** nunique, mode     

**category_1** bool化，mean，count               
**installments** 'sum', 'mean', 'max', 'min', 'std'           
**category_3** 展开之后，mean  
**category_2** 展开之后，mean  
**month_lag**        
    首先创建**month_min**，用来衡量首次使用距离统计起始日的月份，        
    **month_max**,用来衡量最后一次使用距离统计起始日的月份，         
    **month_ptp**,用来衡量月份的跨度       
    **month_std**,用来衡量月份的方差     
    **month_diff**
**purchase_amount** 'sum', 'mean', 'max', 'min', 'std'           
**trans_purchase_amount = np.log(purchase_amount+0.75)** 'sum', 'mean', 'max', 'min', 'std'            
**purchase_date** 从中分离出来 dayofweek mode std，      
                  month std         
        

 
    

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import mode
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/root/code/elo/Tools/')
from tools import *

In [2]:
new = pd.read_csv('/root/data/new_merchant_transactions.csv')
his = pd.read_csv('/root/data/historical_transactions.csv')
train = pd.read_csv('/root/data/train.csv')
test = pd.read_csv('/root/data/test.csv')

In [3]:
def cal_mode(serise):
    return serise.mode()[0]

In [4]:
# authorized_flag


total = pd.concat([his, new]).reset_index(drop=True)
total.category_2.fillna(0, inplace=True)
total.category_3.fillna('D', inplace=True)
total.merchant_id.fillna('NaN', inplace=True)


total['authorized_flag'] = total.authorized_flag.map({'N':0,'Y':1})
auth = total[total['authorized_flag'] == 1]
unauth = total[total['authorized_flag'] == 0]


agg = {
    'authorized_flag':['count', 'mean', 'sum']
}
auth_flag = total.groupby('card_id').agg(agg)
auth_flag.columns = ['_'.join(col).strip() for col in auth_flag.columns.values]
auth_flag.reset_index(inplace=True)

In [5]:
del total, his, new

In [6]:
def agg_dataframe(df):
    
    df['category_1'] = df.category_1.map({'N':0, 'Y':1})
    df = pd.get_dummies(df, columns=['category_2', 'category_3'])
    df['log_amount'] = np.log(df.purchase_amount+0.75)
    
    df['purchase_date'] = pd.to_datetime(df.purchase_date)
    df['month_diff'] = ((datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['dayofweek'] = df.purchase_date.apply(lambda t: t.dayofweek)
    df['month'] = df.purchase_date.apply(lambda t: t.month)
    
    agg = {
        'city_id': ['nunique', cal_mode],
        'category_1': ['mean', 'count'],
        'merchant_category_id': ['nunique', cal_mode],
        'merchant_id': ['nunique', cal_mode],
        'state_id': ['nunique', cal_mode],
        'subsector_id': ['nunique', cal_mode],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'category_2_0.0': ['mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'category_3_D': ['mean'],
        'month_lag': ['min', 'max', np.ptp, 'std'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'log_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'month_diff': ['mean'],
        'dayofweek': [cal_mode, 'std'],
        'month': ['std']
    }
    agg_df = df.groupby(['card_id']).agg(agg)
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
    agg_df.reset_index(inplace=True)
    
    return agg_df

In [7]:
agg_auth = agg_dataframe(auth)
agg_auth.columns = ['auth_' + c if c != 'card_id' else c for c in agg_auth.columns]
agg_unauth = agg_dataframe(unauth)
agg_unauth.columns = ['unauth_' + c if c != 'card_id' else c for c in agg_unauth.columns]

In [10]:
def per_month(df):
    df['log_amount'] = np.log(df.purchase_amount+0.75)
    agg = {
        'purchase_amount': ['count', 'sum', 'mean', 'max', 'min', 'std'],
        'installments': ['count', 'sum', 'mean', 'max', 'min', 'std'],
        'log_amount': ['sum', 'mean', 'max', 'min', 'std']
    }
    df_month = df.groupby(['card_id', 'month_lag']).agg(agg)
    df_month.columns = ['_'.join(col).strip() for col in df_month.columns.values]
    df_month.reset_index(inplace=True)
    
    df_final = df_month.groupby('card_id').agg(['mean', 'std'])
    df_final.columns = ['_'.join(col).strip() for col in df_final.columns.values]
    df_final.reset_index(inplace=True)
    
    return df_final

In [11]:
auth_final = per_month(auth)
auth_final.columns = ['final_auth_' + c if c != 'card_id' else c for c in auth_final.columns]
unauth_final = per_month(auth)
unauth_final.columns = ['final_unauth_' + c if c != 'card_id' else c for c in unauth_final.columns]

In [12]:
train = pd.merge(train, auth_flag, on='card_id', how='left')
train = pd.merge(train, agg_auth, on='card_id', how='left')
train = pd.merge(train, agg_unauth, on='card_id', how='left')
train = pd.merge(train, auth_final, on='card_id', how='left')
train = pd.merge(train, unauth_final, on='card_id', how='left')

In [13]:
test = pd.merge(test, auth_flag, on='card_id', how='left')
test = pd.merge(test, agg_auth, on='card_id', how='left')
test = pd.merge(test, agg_unauth, on='card_id', how='left')
test = pd.merge(test, auth_final, on='card_id', how='left')
test = pd.merge(test, unauth_final, on='card_id', how='left')

In [23]:
for i in train.columns:
    if train[i].dtype == object:
        print(i)

first_active_month
card_id
auth_merchant_id_cal_mode
unauth_merchant_id_cal_mode


In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Columns: 170 entries, first_active_month to final_unauth_log_amount_std_std
dtypes: float64(144), int64(22), object(4)
memory usage: 161.3+ MB


In [24]:
train.to_csv('/root/tempfile/train_final.csv')
test.to_csv('/root/tempfile/test_final.csv')